In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
## load data（input_data/）
mnist = input_data.read_data_sets('input_data/',one_hot=True,reshape=False)
X = tf.placeholder(tf.float32,[None,28,28,1], name="X") # input of the model
Y_ = tf.placeholder(tf.float32,[None,10], name="Y_") # output of the model

Extracting input_data/train-images-idx3-ubyte.gz
Extracting input_data/train-labels-idx1-ubyte.gz
Extracting input_data/t10k-images-idx3-ubyte.gz
Extracting input_data/t10k-labels-idx1-ubyte.gz


In [3]:
## Parameters initialization
K = 16
L = 32
M = 32

W1 = tf.Variable(tf.truncated_normal([5,5,1,K],stddev=0.1))
B1 = tf.Variable(tf.constant(0.1,tf.float32,[K]))
W2 = tf.Variable(tf.truncated_normal([5,5,K,L],stddev=0.1))
B2 = tf.Variable(tf.constant(0.1,tf.float32,[L]))
W3 = tf.Variable(tf.truncated_normal([7*7*L,M],stddev=0.1))
B3 = tf.Variable(tf.constant(0.1,tf.float32,[M]))
W4 = tf.Variable(tf.truncated_normal([M,10],stddev=0.1))
B4 = tf.Variable(tf.constant(0.1,tf.float32,[10]))

In [4]:
## Model structure
keep_prob = tf.placeholder(tf.float32, name="keep_prob") # dropout: keep prob

conv1 = tf.nn.relu(tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')+B1)
pool1 = tf.nn.max_pool(conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
conv2 = tf.nn.relu(tf.nn.conv2d(pool1,W2,strides=[1,1,1,1],padding='SAME')+B2)
pool2 = tf.nn.max_pool(conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
pool2_flat = tf.reshape(pool2,[-1,7*7*L])
fc1 = tf.nn.relu(tf.matmul(pool2_flat,W3)+B3)
fc1_drop = tf.nn.dropout(fc1,keep_prob)
Ylogits = tf.matmul(fc1_drop,W4)+B4
Y = tf.nn.softmax(Ylogits, name="Y")

In [5]:
## Loss function: cross entropy 
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits,labels=Y_)
cross_entropy = tf.reduce_mean(cross_entropy)

In [6]:
## Accuracy
is_accuracy = tf.equal(tf.argmax(Y_,1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(is_accuracy,tf.float32))

In [7]:
## Model training
epoch = 100
batch = 100

# some lists used for saving results
train_acc = []
train_loss = []
test_acc = []
test_loss = []

lr = tf.placeholder(tf.float32) # learning rate (variable)
optimizer = tf.train.AdamOptimizer(lr) # optimal method
train_step = optimizer.minimize(cross_entropy)
saver = tf.train.Saver()
init = tf.global_variables_initializer() # initialize all variables

with tf.Session() as sess:
    sess.run(init) # run initialization process
    for i in range(epoch):
        # learning rate
        max_lr = 0.003
        min_lr = 0.0001
        decay_speed = 2000.0
        learning_rate = min_lr+(max_lr-min_lr)*np.math.exp(-i/decay_speed)

        # training step
        batch_X,batch_Y = mnist.train.next_batch(batch) # load one batch training data
        train_data = {X:batch_X, Y_:batch_Y,lr:learning_rate, keep_prob:0.5} # dictionary
        sess.run(train_step,feed_dict=train_data) # train one step

        # save results
        acc,loss = sess.run([accuracy,cross_entropy],feed_dict=train_data)
        train_acc.append(acc)
        train_loss.append(loss)

        test_data = {X:mnist.test.images,Y_:mnist.test.labels,keep_prob:1}
        acc,loss = sess.run([accuracy,cross_entropy],feed_dict=test_data)
        test_acc.append(acc)
        test_loss.append(loss)

        # print training process
        if i%10 == 0:
            print("epoch = %d, " %i, "test accuracy = %.4f," %test_acc[i], \
            "test loss = %.6f" %test_loss[i], "learning rate = %.6f" %learning_rate)
        if i == epoch-1:
            saver.save(sess, 'test_model/test_model')

print("test accuracy = %.4f, " %test_acc[-1], "test loss = %.6f" %test_loss[-1])

('epoch = 0, ', 'test accuracy = 0.1128,', 'test loss = 2.333412', 'learning rate = 0.003000')
('epoch = 10, ', 'test accuracy = 0.5641,', 'test loss = 2.042782', 'learning rate = 0.002986')
('epoch = 20, ', 'test accuracy = 0.6573,', 'test loss = 1.344129', 'learning rate = 0.002971')
('epoch = 30, ', 'test accuracy = 0.7656,', 'test loss = 0.968288', 'learning rate = 0.002957')
('epoch = 40, ', 'test accuracy = 0.8053,', 'test loss = 0.699313', 'learning rate = 0.002943')
('epoch = 50, ', 'test accuracy = 0.8371,', 'test loss = 0.595425', 'learning rate = 0.002928')
('epoch = 60, ', 'test accuracy = 0.8725,', 'test loss = 0.485973', 'learning rate = 0.002914')
('epoch = 70, ', 'test accuracy = 0.8955,', 'test loss = 0.445268', 'learning rate = 0.002900')
('epoch = 80, ', 'test accuracy = 0.9055,', 'test loss = 0.364110', 'learning rate = 0.002886')
('epoch = 90, ', 'test accuracy = 0.9173,', 'test loss = 0.304880', 'learning rate = 0.002872')
('test accuracy = 0.9232, ', 'test loss =